# Housing rent prices and venues data analysis of Frankfurt

#### Import all the packages

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip -q install folium
import folium # map rendering library

!pip -q install plotly

# to retrieve the districts coordinates
import urllib.parse

print('Libraries imported.')

Libraries imported.


## Data preparation

### Retrieve list of Frankfurt's districts from Wikipedia's page

In [81]:
# Webpage url (old link as suggested in the Discussion Forums)                                                                                                             
url = 'https://de.wikipedia.org/wiki/Liste_der_Stadtteile_von_Frankfurt_am_Main'

# Extract tables using pandas
dfs = pd.read_html(url,header=[0],flavor='bs4')

# Get first table                                                                                                           
df = dfs[0]

df

,Nr.,Stadtteil,Fläche[3]in km²,Einwohner,Weiblich,Männlich,Deutsche,Ausländer,Ausländerin Prozent,Einwohnerje km²,Ortsbezirk,Stadtgebietseit,Vorherige Zugehörigkeit
0,1,Altstadt,506,4.218,2.065,2.153,2.669,1.549,367,8336,01 Innenstadt I,1866[Anm. 1],Freie Stadt Frankfurt
1,2,Innenstadt,1491,6.599,3.112,3.487,3.539,3.060,464,4426,01 Innenstadt I,1866[Anm. 2],Freie Stadt Frankfurt
2,3,Bahnhofsviertel,542,3.552,1.321,2.231,1.706,1.846,52,6554,01 Innenstadt I,1866[Anm. 3],Freie Stadt Frankfurt
3,4,Westend-Süd,2497,19.314,9.839,9.475,14.006,5.308,275,7735,02 Innenstadt II,1866[Anm. 3],Freie Stadt Frankfurt
4,5,Westend-Nord,1632,10.373,5.391,4.982,7.366,3.007,29,6356,02 Innenstadt II,1866[Anm. 3],Freie Stadt Frankfurt
5,6,Nordend-West,3100,30.897,15.845,15.052,24.144,6.753,219,9967,03 Innenstadt III,1866[Anm. 3],Freie Stadt Frankfurt
6,7,Nordend-Ost,1532,23.182,12.045,11.137,18.016,5.166,223,15132,03 Innenstadt III,1866[Anm. 3],Freie Stadt Frankfurt
7,8,Ostend,5564,29.477,15.108,14.369,21.070,8.407,285,5298,04 Bornheim/Ostend,1866[Anm. 3],Freie Stadt Frankfurt
8,9,Bornheim,2786,30.917,16.316,14.601,23.511,7.406,24,11097,04 Bornheim/Ostend,1877,Stadtkreis Frankfurt am Main[Anm. 4]
9,10,Gutleutviertel,1792,6.964,3.047,3.917,4.000,2.964,426,3886,01 Innenstadt I,1866[Anm. 3],Freie Stadt Frankfurt


In [122]:
df_districts = df.iloc[0:-1,1:8].reset_index(drop=True) # Remove the last row which is the total
df_districts.columns = ['District', 'Surface_km2',"population", "female_pop", "male_pop", "german_pop", "foreign_pop"]

#Fix Flughafen value
df_districts.loc[df_districts.District == "Flughafen", ["population","female_pop", "male_pop", "german_pop", "foreign_pop"]] = df_districts.loc[df_districts.District == "Sachsenhausen-Süd", ["population","female_pop", "male_pop", "german_pop", "foreign_pop"]].values.tolist()

df_districts[["population","female_pop","male_pop","german_pop","foreign_pop"]] = df_districts[["population","female_pop","male_pop","german_pop","foreign_pop"]].apply(lambda x: x.str.replace('.',''))
df_districts = df_districts.astype({'population': 'int64', 'female_pop': 'int64', 'male_pop': 'int64', 'german_pop': 'int64', 'foreign_pop': 'int64'})

df_districts

,District,Surface_km2,population,female_pop,male_pop,german_pop,foreign_pop
0,Altstadt,506,4218,2065,2153,2669,1549
1,Innenstadt,1491,6599,3112,3487,3539,3060
2,Bahnhofsviertel,542,3552,1321,2231,1706,1846
3,Westend-Süd,2497,19314,9839,9475,14006,5308
4,Westend-Nord,1632,10373,5391,4982,7366,3007
5,Nordend-West,3100,30897,15845,15052,24144,6753
6,Nordend-Ost,1532,23182,12045,11137,18016,5166
7,Ostend,5564,29477,15108,14369,21070,8407
8,Bornheim,2786,30917,16316,14601,23511,7406
9,Gutleutviertel,1792,6964,3047,3917,4000,2964


## Get the coordinates of the districts

In [127]:
df_districts["Latitude"] = 0.00000
df_districts["Longitude"] = 0.00000

for row, district in enumerate(df_districts["District"]):
    downloaded = False
    
    while downloaded == False:
        try:
            print("Checking "+district)
            address = district+', Frankfurt am Main, Germany'
            url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
            #print(url)

            response = requests.get(url).json()
            df_districts["Latitude"][row] = round(float((response[0]["lat"])), 5)
            df_districts["Longitude"][row] = round(float((response[0]["lon"])), 5)
            downloaded = True
        except:
              print("Error with district download - trying again")

#df_districts

Checking Altstadt


<ipython-input-127-7f58b260a939>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_districts["Latitude"][row] = round(float((response[0]["lat"])), 5)
<ipython-input-127-7f58b260a939>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_districts["Longitude"][row] = round(float((response[0]["lon"])), 5)


Checking Innenstadt
Checking Bahnhofsviertel
Error with district download - trying again
Checking Bahnhofsviertel
Checking Westend-Süd
Error with district download - trying again
Checking Westend-Süd
Error with district download - trying again
Checking Westend-Süd
Error with district download - trying again
Checking Westend-Süd
Error with district download - trying again
Checking Westend-Süd
Error with district download - trying again
Checking Westend-Süd
Checking Westend-Nord
Checking Nordend-West
Checking Nordend-Ost
Checking Ostend
Checking Bornheim
Error with district download - trying again
Checking Bornheim
Checking Gutleutviertel
Checking Gallus
Checking Bockenheim
Checking Sachsenhausen-Nord
Checking Sachsenhausen-Süd
Checking Flughafen
Checking Oberrad
Checking Niederrad
Checking Schwanheim
Checking Griesheim
Checking Rödelheim
Error with district download - trying again
Checking Rödelheim
Checking Hausen
Error with district download - trying again
Checking Hausen
Error with d

## Download data on Rental price index 

In [124]:
# Webpage url (old link as suggested in the Discussion Forums)                                                                                                             
url = 'https://www.miet-check.de/stadtteile_uebersicht.php?stadt=Frankfurt%20am%20Main'

# Extract tables using pandas
dfs = pd.read_html(url,header=[0],flavor='bs4')

# Get first table                                                                                                           
df = dfs[0]

district_rent_price = pd.DataFrame(columns =["District", "Rent_per_sqm"])

district_rent_price = district_rent_price.append(df[["Stadtteile in","Stadtteile in.1"]].rename(columns={"Stadtteile in": "District", "Stadtteile in.1": "Rent_per_sqm"}), ignore_index=True)
district_rent_price = district_rent_price.append(df[["Stadtteile in.2","Stadtteile in.3"]].rename(columns={"Stadtteile in.2": "District", "Stadtteile in.3": "Rent_per_sqm"}), ignore_index=True)
district_rent_price = district_rent_price.append(df[["Stadtteile in.4","Stadtteile in.5"]].rename(columns={"Stadtteile in.4": "District", "Stadtteile in.5": "Rent_per_sqm"}), ignore_index=True)
district_rent_price["Rent_per_sqm"] = district_rent_price["Rent_per_sqm"].str.replace(",", ".").str.replace(" €", "").astype(float)

pd.set_option('display.max_rows', None)

#Price split for Sachsenhausen Nord and Sud not available 
district_rent_price.loc[district_rent_price.District == "Sachsenhausen", "District"] = "Sachsenhausen-Süd"
district_rent_price.loc[45] = ["Sachsenhausen-Nord", float(district_rent_price.loc[district_rent_price.District == "Sachsenhausen-Süd"].Rent_per_sqm)]
#Altstadt price same as Innenstadt
district_rent_price.loc[46] = ["Altstadt", float(district_rent_price.loc[district_rent_price.District == "Innenstadt"].Rent_per_sqm)]

# Merge back to original df
df_districts = pd.merge(df_districts, district_rent_price, on="District")

In [128]:
df_districts

,District,Surface_km2,population,female_pop,male_pop,german_pop,foreign_pop,Latitude,Longitude
0,Altstadt,506,4218,2065,2153,2669,1549,50.11044,8.68235
1,Innenstadt,1491,6599,3112,3487,3539,3060,50.11456,8.68359
2,Bahnhofsviertel,542,3552,1321,2231,1706,1846,50.10841,8.66815
3,Westend-Süd,2497,19314,9839,9475,14006,5308,50.11524,8.66227
4,Westend-Nord,1632,10373,5391,4982,7366,3007,50.12636,8.66792
5,Nordend-West,3100,30897,15845,15052,24144,6753,50.12491,8.67795
6,Nordend-Ost,1532,23182,12045,11137,18016,5166,50.12492,8.69232
7,Ostend,5564,29477,15108,14369,21070,8407,50.11237,8.69997
8,Bornheim,2786,30917,16316,14601,23511,7406,50.12973,8.71061
9,Gutleutviertel,1792,6964,3047,3917,4000,2964,50.10171,8.66054


## Create a map of Frankfurt with neighborhoods superimposed on top


In [140]:
address = 'Frankfurt am Main, DE'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Frankfurt are {}, {}.'.format(latitude, longitude))

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Frankfurt+am+Main%2C+DE&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [132]:
import folium

# file name - file is located in the working directory
with urlopen('https://offenedaten.frankfurt.de/dataset/85b38876-729c-4a78-910c-a52d5c6df8d2/resource/84dff094-ab75-431f-8c64-39606672f1da/download/ffmstadtteilewahlen.geojson') as response:
    districts = json.load(response)
    
communities_geo = districts # geojson file

# create a plain Frankfur map
communities_map = folium.Map(location=[latitude, longitude], zoom_start=10)

#generate choropleth map 
folium.Choropleth(
    geo_data=communities_geo,
    data=df_districts[['District', "Rent_per_sqm"]],
    columns=['District', 'Rent_per_sqm'],
    key_on='feature.properties.STTLNAME',
    fill_color='YlGnBu', 
    fill_opacity=1, 
    line_opacity=1,
    legend_name='Rent per sqm',
    smooth_factor=0).add_to(communities_map)

# display map
#communities_map
# add markers to map
for lat, lng, district in zip(df_districts['Latitude'], df_districts['Longitude'], df_districts['Stadtteil']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(communities_map)  


style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    districts,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['STTLNAME'],
        aliases=['Neighborhood'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)

communities_map.add_child(NIL)
#communities_map.keep_in_front(NIL)
folium.LayerControl().add_to(communities_map)
communities_map

NameError: name 'urlopen' is not defined